In [1]:
import numpy as np
from sklearn.externals import joblib
import copy
import random
import math
import matplotlib as mp
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.base import clone
from sklearn import metrics

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
#mp.rcParams.update({'font.size': 22})

In [4]:
datadict = joblib.load("Two_Port_Standard_Dictionary_Cleaned_Interpolated.pkl")

In [5]:
device = "CTN210"
system = "System 2,7"

In [6]:
num_fake_points = 15000
magS11_deviation = 0.003
magS21_deviation = 0.0001
magS22_deviation = 0.003
frequency_range = np.linspace(2,18,50)

In [7]:
magS11_points = []
magS21_points = []
magS22_points = []
for i in range(num_fake_points):
    new_dataset = copy.deepcopy(datadict[device][system][random.choice(datadict[device][system].keys())])
    new_dataset["magS11"] += np.random.normal(0,magS11_deviation,50)
    new_dataset["magS21"] += np.random.normal(0,magS21_deviation,50)
    new_dataset["magS22"] += np.random.normal(0,magS22_deviation,50)
    magS11_points.append(new_dataset["magS11"].tolist())
    magS21_points.append(new_dataset["magS21"].tolist())
    magS22_points.append(new_dataset["magS22"].tolist())
magS11_arr = np.array(magS11_points)
magS21_arr = np.array(magS21_points)
magS22_arr = np.array(magS22_points)

In [18]:
estimator_range = np.arange(1,20,1)
randomness_amplitudes = np.linspace(0.0003,0.008,33)
red_vals = ["#" + hex(c)[2:-1] if len(hex(c)[2:-1])>=2 else "#0" + hex(c)[2:-1] for c in np.append(np.arange(0,256,8),255)]
green_vals = [hex(255-c)[2:-1] if len(hex(255-c)[2:-1])>=2 else "0" + hex(255-c)[2:-1] for c in np.append(np.arange(0,256,8),255)]
color_vals = [r+g+"00" for r,g in zip(red_vals,green_vals)]
print color_vals

 ['#00ff00', '#08f700', '#10ef00', '#18e700', '#20df00', '#28d700', '#30cf00', '#38c700', '#40bf00', '#48b700', '#50af00', '#58a700', '#609f00', '#689700', '#708f00', '#788700', '#807f00', '#887700', '#906f00', '#986700', '#a05f00', '#a85700', '#b04f00', '#b84700', '#c03f00', '#c83700', '#d02f00', '#d82700', '#e01f00', '#e81700', '#f00f00', '#f80700', '#ff0000']


In [19]:
def get_magS21_f1_score(randomness_amplitude,estimatornum):
    #Making Bad Data
    incorrect_magS21_data = copy.deepcopy(magS21_arr)
    for i in range(len(incorrect_magS21_data)):
        incorrect_magS21_data[i] = (incorrect_magS21_data[i] + np.random.normal(0,randomness_amplitude,50))
    #Preparing Data for training
    S21_training_data = []
    S21_training_data.extend(incorrect_magS21_data.tolist())
    S21_training_data.extend(magS21_arr.tolist())
    S21_training_data_arr = np.array(S21_training_data)
    targets = ["not valid","valid"]
    S21_target_arr = np.repeat(targets,len(magS21_arr))
    S21_training_data_arr,S21_target_arr = shuffle(S21_training_data_arr,S21_target_arr, random_state=0)
        #print "Made training and test data!"
    #Cloning and Training Classifier
    clf = RandomForestClassifier(n_estimators = estimatornum)
        #print "Cloned Classifier!"
    clf.fit(S21_training_data_arr[:len(magS21_arr)],S21_target_arr[:len(magS21_arr)])
        #print "Trained Classifier!"
    #Making Predictions
    expected = S21_target_arr[len(magS21_arr):]
    predicted = clf.predict(S21_training_data_arr[len(magS21_arr):])
        #print "Did Classifier Predictions!"
    return metrics.f1_score(expected, predicted, average="macro")

In [33]:
import sys
graph_data = []
j=1
print "Progress:",
amount_progress = "0 / " + str(len(randomness_amplitudes))
print amount_progress,
for index in range(len(randomness_amplitudes)):
    amp=randomness_amplitudes[index]
    data = []
    for i in range(len(estimator_range)):
        f1score = get_magS21_f1_score(amp,estimator_range[i])
        data.append(f1score)
    graph_data.append((amp,data,color_vals[index]))
    amount_progress_new = str(j) + " / " + str(len(randomness_amplitudes))
    print amount_progress_new
    sys.stdout.flush()
    amount_progress = amount_progress_new
    j+=1
print ("\b" * len(amount_progress)) + ("\b" * len("Progress:")) + "\bAll Amplitudes Completed!"

 Progress: 0 / 33 1 / 33


KeyboardInterrupt: 

In [ ]:
print graph_data[0]

In [ ]:
plt.figure(figsize=(10,10))
graph_handles = []
for index in range(len(graph_data)):
    fcolor=graph_data[index][2]
    graph_handles.extend(
        plt.plot(estimator_range,graph_data[index][1],label=str(graph_data[index][0]),color=fcolor))
    if index+1 < len(graph_data):
        plt.fill_between(estimator_range,graph_data[index][1],graph_data[index+1][1],facecolor=fcolor)
plt.legend(handles=graph_handles,bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title="Randomness Amplitudes")
plt.tight_layout()
plt.show()